In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import random
from random import choices
from scipy import stats
import pickle
from scipy.stats import chi2_contingency
import imblearn
from imblearn.under_sampling import TomekLinks
from ctgan import CTGAN
import torch
import sdmetrics
from sdmetrics.reports.single_table import QualityReport
from sdmetrics.single_column import TVComplement, KSComplement
import numpy as np

# Load data

In [ ]:
traininput = pd.read_csv("data/ltc_events_readmission_merged_admimeth_labels_final_train_nopostadmission.tsv",sep='\t')
trainlabels = pd.read_csv("data/ltc_events_readmission_merged_admimeth_inputs_final_train.tsv",sep='\t')
print(traininput.shape, trainlabels.shape)

(55621, 21) (1236075, 204)


# Initial EDA

In [ ]:
# Smoking
plt.figure(figsize = (12,2))
b = traininput.smoking_status.hist()

In [ ]:
# Checking for errors in delta_length column
dftest = traininput.copy()

dftest['first_discharge_date'] = pd.to_datetime(dftest['first_discharge_date'])
dftest['second_admi_date'] = pd.to_datetime(dftest['second_admi_date'])
dftest['Difference'] = (dftest['second_admi_date'] -
                        dftest['first_discharge_date']).dt.days

a = dftest['Difference']-traininput['delta_length']

print('Number of erroneous rows: ', a.shape[0] - sum(a == 0))
print('Max error:', max(a))
plt.figure(figsize=(4, 2))
a.hist(bins=1000)
plt.ylim(0, 3000)
plt.xlim(1, 50)

In [ ]:
# Checking for error in target_feature
traininput['readmission_state'][traininput['delta_length'] <= 30].value_counts()
traininput['readmission_state'][traininput['delta_length'] < 30].value_counts()
traininput['readmission_state'][traininput['delta_length'] > 30].value_counts()

plt.figure(figsize=(4, 2))
sns.histplot(data=traininput[traininput['delta_length']
             < 100], x="delta_length", hue="readmission_state")
plt.xlim(0,)

In [ ]:
# Checking for errors in delta_time column
dftest = trainlabels.copy()
dftest['event_date'] = pd.to_datetime(dftest['event_date'])
dftest.sort_values(['patient_id', 'event_date'], inplace=True)
dftest['days_since_previous'] = dftest.groupby('patient_id')['event_date'].diff().dt.days
dftest['days_since_previous'].fillna(0, inplace=True)

a = dftest['days_since_previous']-trainlabels['delta_time']

print('Number of erroneous rows: ', a.shape[0] - sum(a == 0))
print('Max error:', max(a))

# DATASET CURATION

## Fix LTCs

In [ ]:
# Modify to 1 all floats bigger than 0.
trainlabels.iloc[:,3:] = np.ceil(trainlabels.iloc[:,3:].values)

## Binarize prescriptions

In [ ]:
# Historical
histpresc = traininput['historical_scripts'].str.cat(sep=',')
histpresc_list = re.split(',\s*', histpresc)
unique_histpresc = list(set(histpresc_list))

for word in unique_histpresc:
    name = 'hist_' + word
    traininput[name] = traininput['historical_scripts'].str.contains(
        word).astype(int)

# Preadmission
pread = traininput['preadmission_scripts'].str.cat(sep=',')
pread_list = re.split(',\s*', pread)
unique_pread = list(set(pread_list))

for word in unique_pread:
    name = 'pread_' + word
    traininput[name] = traininput['preadmission_scripts'].str.contains(
        word).astype(int)

traininput.drop('preadmission_scripts', axis = 1, inplace = True)
traininput.drop('historical_scripts', axis = 1, inplace = True)

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['historical_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

...

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  traininput[name] = traininput['preadmission_scripts'].str.contains(
C:\Users\hugoc\AppData\Local\Temp\ipykernel_8308\1141991334.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert`

## Merge binarized prescriptions & LTC BOWs

In [ ]:
sorted_ltcs = trainlabels.sort_values('event_date', ascending=False).groupby('patient_id').head(1)

In [ ]:
ltc_columns = sorted_ltcs.iloc[:, 3:]
prefixed_ltcs = ltc_columns.add_prefix('ltc_')

In [ ]:
mergeddf = traininput.join(prefixed_ltcs, on='eid')

## Binarize diags

In [ ]:
diag_dict = pickle.load( open("top_250_merged_diags.pkl", "rb") ) # The path to open this .pkl file may need to be changed

one_hot_encoded = pd.get_dummies(mergeddf['diag'])

# Only top 250 diags
keys = diag_dict.keys()
filtered_df = one_hot_encoded[[col for col in one_hot_encoded.columns if col in keys]]
prefixed_df = filtered_df.add_prefix('diag_')

mergeddf = pd.concat([mergeddf, prefixed_df], axis=1)
mergeddf.drop('diag', axis = 1, inplace = True)

## Define variable metada

In [ ]:
admission = list(mergeddf.columns)[:7]

target = list(mergeddf.columns)[7]

demographics = list(mergeddf.columns)[8:18]
hist_drugs =  list(mergeddf.columns)[18:1054-540]
pread_drugs = list(mergeddf.columns)[1054-540:1054-202]

ltcs = list(mergeddf.columns)[1054-202:1053]
diags = list(mergeddf.columns)[1053:]

In [ ]:
# list
print(admission[0],admission[-1])
print(target)
print(demographics[0],demographics[-1])
print(hist_drugs[0],hist_drugs[-1])
print(pread_drugs[0],pread_drugs[-1])
print(ltcs[0],ltcs[-1])
print(diags[0],diags[-1])

eid delta_length
readmission_state
sex body_fat_percentage
hist_9003020 hist_1906010
pread_1404150 pread_0706000
ltc_PMR ltc_SARS_CoV2
diag_A099 diag_Z466


In [ ]:
binary = ['sex'] + hist_drugs + pread_drugs + ltcs + diags
target = target
continuous = ['age_at_event', 'ltc_count', 'townsend_deprivation_index', 'alcohol_intake_frequency', 'smoking_status',
              'relative_smoking_pack_years', 'ethnic_background', 'BMI', 'body_fat_percentage', 'first_admi_duration']

In [ ]:
toremove = ['eid', 'first_admi_date','first_discharge_date', 'second_admi_date', 'delta_length', 'admi_meth']

## Deal missing values

In [ ]:
print('Number of NaNs:', mergeddf.isna().sum().sum())

Number of NaNs: 11155701


In [ ]:
columns_with_nan = mergeddf.columns[mergeddf.isna().any()].tolist()
mergeddf[columns_with_nan] = mergeddf[columns_with_nan].fillna(0)

## Transform gender column

In [ ]:
# Male = 0, Female = 1
mapping = {'M': 0, 'F': 1}
mergeddf['sex'] = mergeddf['sex'].map(mapping)

In [ ]:
mergeddf.to_excel('data/data_fixedltcs.xlsx',index = False )